In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import numpy as np

In [187]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,f1_score,confusion_matrix

In [298]:
df = pd.read_excel("CAT_Training_Dataset_V3 File.xlsx",engine='openpyxl')

In [301]:
df = df.drop(df[df["date"].apply(lambda x:str(x).isnumeric()== False)]["ID"])

In [302]:
df["units"] = df["units"].apply(lambda x:int(x))

In [303]:
df[df["svrty_level"].apply(lambda x:(x>=1 and x<=3)==False)]
df = df.drop(512365)

In [304]:
day1 = datetime.datetime.fromisoformat('2021-01-01T00:00:00')
df["DateDiff"]=df["date"].apply(lambda x:int(x)/1440)

In [325]:
df["event"]=df["event"].astype('category')
df["eve_coded"]=df["event"].cat.codes
df["candidate_id"]=df["candidate"].apply(lambda x:int(x.replace("ABCDE","")))
# df.drop(["candidate","event"],axis = 1,inplace = True)

In [513]:
x = df[["candidate_id","units","svrty_level","DateDiff"]]

In [516]:
threshold = x.groupby("candidate_id",as_index=True)["units"].max()+(x.groupby("candidate_id",as_index=True)["units"].std()*0.5)

In [501]:
pd.DataFrame(threshold).to_csv("threshold.csv")

In [520]:
XD = x.drop(["DateDiff"],axis=1)
yd = x["DateDiff"]

In [521]:
XD_train,XD_test,yd_train,yd_test = train_test_split(XD,yd,test_size = 0.40)

In [523]:
sc_X = StandardScaler()
XD_train = sc_X.fit_transform(XD_train)
XD_test = sc_X.transform(XD_test)

In [524]:
classifier = LinearRegression()
classifier = classifier.fit(XD_train,yd_train)

In [525]:
y_pred = classifier.predict(XD_test)


In [528]:
import pickle
pickle.dump(classifier,open("catathon.sav", "wb"))
pickle.dump(sc_X, open("scaler.sav", "wb"))